# Data Collection

This notebook aims to collect historical price data for stocks in the S&P 500 from the **yfinance** module.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import statsmodels.tsa.vector_ar as ar
import datetime
from dateutil.relativedelta import relativedelta
import itertools
import math

c:\Users\nayrb\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Get symbols of Current Stocks in S&P500 and download data

In [2]:
payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
symbols = first_table['Symbol'].values.tolist()

# Update symbols to correct for yfinance accessibility
symbols = list(map(lambda x: x.replace('BRK.B', 'BRK-B'), symbols))
symbols = list(map(lambda x: x.replace('BF.B', 'BF-B'), symbols))

symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',


In [3]:
# Download historical stock data based on the start and end dates
data = yf.download(symbols, start = "2015-01-01", end = "2025-01-01", interval="1d")

Failed to get ticker 'CNC' reason: Expecting value: line 1 column 1 (char 0)
[                       0%                       ]Failed to get ticker 'FI' reason: Expecting value: line 1 column 1 (char 0)
Failed to get ticker 'DHI' reason: Expecting value: line 1 column 1 (char 0)
[                       0%                       ]  2 of 503 completedFailed to get ticker 'NI' reason: Expecting value: line 1 column 1 (char 0)
[                       1%                       ]  4 of 503 completedFailed to get ticker 'HSY' reason: Expecting value: line 1 column 1 (char 0)
[                       1%                       ]  5 of 503 completedFailed to get ticker 'APO' reason: Expecting value: line 1 column 1 (char 0)
Failed to get ticker 'WDC' reason: Expecting value: line 1 column 1 (char 0)
[                       1%                       ]  6 of 503 completedFailed to get ticker 'WTW' reason: Expecting value: line 1 column 1 (char 0)
Failed to get ticker 'D' reason: Expecting value: line 1

KeyboardInterrupt: 

Failed to get ticker 'EXE' reason: Expecting value: line 1 column 1 (char 0)
[*****************     35%                       ]  176 of 503 completedFailed to get ticker 'IFF' reason: Expecting value: line 1 column 1 (char 0)
[*****************     35%                       ]  177 of 503 completedFailed to get ticker 'DUK' reason: Expecting value: line 1 column 1 (char 0)
[*****************     35%                       ]  178 of 503 completedFailed to get ticker 'ADSK' reason: Expecting value: line 1 column 1 (char 0)
Failed to get ticker 'SBAC' reason: Expecting value: line 1 column 1 (char 0)
[*****************     36%                       ]  179 of 503 completedFailed to get ticker 'CCI' reason: Expecting value: line 1 column 1 (char 0)
[*****************     36%                       ]  181 of 503 completedFailed to get ticker 'ZBH' reason: Expecting value: line 1 column 1 (char 0)
Failed to get ticker 'MOS' reason: Expecting value: line 1 column 1 (char 0)
[*****************    

Clean data for any timeseries that was missing

In [9]:
import pandas as pd
close_data = pd.read_csv('./data/Data.csv')


In [5]:
close_data.dropna(axis=1, inplace=True)

close_data.head()

C:\Users\nayrb\AppData\Local\Temp\ipykernel_2164\561113609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_data.dropna(axis=1, inplace=True)


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,


In [6]:
close_data.to_csv('./data/Data.csv')

In [7]:
stock_tickers = close_data.columns

stock_tickers

Index(['A', 'AAPL', 'ABBV', 'ABNB', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM',
       ...
       'WTW', 'WY', 'WYNN', 'XEL', 'XOM', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZTS'],
      dtype='object', name='Ticker', length=503)

In [14]:
# Calculate daily returns
returns = close_data.pct_change().dropna()

# Calculate annualized returns (252 trading days in a year)
annualized_returns = returns.mean() * 252

# Calculate annualized volatility
annualized_volatility = returns.std() * np.sqrt(252)

# Calculate Sharpe Ratio (assuming risk-free rate of 0 for simplicity)
sharpe_ratio = annualized_returns / annualized_volatility

# Create a DataFrame to store the metrics
performance_metrics = pd.DataFrame({
    'Annualized Return': annualized_returns,
    'Annualized Volatility': annualized_volatility,
    'Sharpe Ratio': sharpe_ratio
})

# Display the top 5 companies by Sharpe Ratio
top_5_sharpe = performance_metrics.sort_values('Sharpe Ratio', ascending=False).head(5)
print("Top 5 Companies by Sharpe Ratio:")
print(top_5_sharpe)

# Create a binary classification target based on Sharpe Ratio
# Companies with Sharpe Ratio above median are classified as "Good"
median_sharpe = performance_metrics['Sharpe Ratio'].median()
performance_metrics['Good'] = (performance_metrics['Sharpe Ratio'] > median_sharpe).astype(int)

# Prepare data for machine learning
X = performance_metrics[['Annualized Return', 'Annualized Volatility']]
y = performance_metrics['Good']

# Ensure data is clean and aligned
X = X.dropna()
y = y[X.index]  # Align y with X's index

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("\nModel Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

# Visualize the results
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 10))

# Plot 1: Scatter plot of returns vs volatility with Sharpe ratio as color
plt.subplot(2, 2, 1)
scatter = plt.scatter(X['Annualized Volatility'], X['Annualized Return'], 
                     c=performance_metrics.loc[X.index, 'Sharpe Ratio'], 
                     cmap='viridis', alpha=0.7)
plt.colorbar(scatter, label='Sharpe Ratio')
plt.xlabel('Annualized Volatility')
plt.ylabel('Annualized Return')
plt.title('Risk-Return Profile of S&P 500 Companies')

# Plot 2: Top 5 companies by Sharpe ratio
plt.subplot(2, 2, 2)
top_5_sharpe['Sharpe Ratio'].sort_values().plot(kind='barh', color='green')
plt.xlabel('Sharpe Ratio')
plt.title('Top 5 Companies by Sharpe Ratio')

# Plot 3: Model prediction visualization
plt.subplot(2, 2, 3)
sns.scatterplot(x='Annualized Volatility', y='Annualized Return', 
                hue=y, data=X.join(y), palette=['red', 'green'], alpha=0.7)
plt.title('Companies Classified by Sharpe Ratio\n(Red: Below Median, Green: Above Median)')

# Plot 4: Feature importance
plt.subplot(2, 2, 4)
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Feature Importance in Predicting "Good" Companies')

plt.tight_layout()
plt.show()

Top 5 Companies by Sharpe Ratio:
     Annualized Return Annualized Volatility Sharpe Ratio
Date               NaN                   NaN          NaN
A                  NaN                   NaN          NaN
AAPL               NaN                   NaN          NaN
ABBV               NaN                   NaN          NaN
ABNB               NaN                   NaN          NaN


c:\Users\nayrb\anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.